In [0]:

# usage of with column : to change value, to change data type, to create new column
data=[(1,'loki','22'),(2,'gopi',25)]
schema=['id','name','age']
df=spark.createDataFrame(data,schema)
df.show()
df.printSchema()

+---+----+---+
| id|name|age|
+---+----+---+
|  1|loki| 22|
|  2|gopi| 25|
+---+----+---+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)



In [0]:
from pyspark.sql.functions import * 
# lit is used to pass value to clumns

In [0]:
# to change data type of column using with column
# IT WILL CREATE NEW DATAFRAME , in spark when we do any altration or udation it's not going to change
df1=df.withColumn('Age',col('age').cast('Integer'))

In [0]:
# To update value in a column
#df2=df1.withColumn('Age',df1.Age+1) # first parameter is new column u want to create , second parm sholud be  column type
df2=df1.withColumn('Age',col('Age')*2)
df2.show()

+---+----+---+
| id|name|Age|
+---+----+---+
|  1|loki| 44|
|  2|gopi| 50|
+---+----+---+



In [0]:
# it will create new column beacuse there is no column with joindate
df3=df1.withColumn('joindate',to_date(lit('2020-10-10'),'yyyy-MM-dd')) #to_date('value of date in str format', the format of that date) we conveted string to date value
df3.show()

+---+----+---+----------+
| id|name|Age|  joindate|
+---+----+---+----------+
|  1|loki| 22|2020-10-10|
|  2|gopi| 25|2020-10-10|
+---+----+---+----------+



In [0]:
# date format is used to convert one fornat of data int oanother format
df3.withColumn('joindate',date_format(col('joindate'),'yyyy/MM/dd')).show()

+---+----+---+----------+
| id|name|Age|  joindate|
+---+----+---+----------+
|  1|loki| 22|2020/10/10|
|  2|gopi| 25|2020/10/10|
+---+----+---+----------+



In [0]:
# rename the existing column
help(df3.withColumnRenamed)

Help on method withColumnRenamed in module pyspark.sql.dataframe:

withColumnRenamed(existing: str, new: str) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` by renaming an existing column.
    This is a no-op if the schema doesn't contain the given column name.
    
    .. versionadded:: 1.3.0
    
    .. versionchanged:: 3.4.0
        Support Spark Connect.
    
    Parameters
    ----------
    existing : str
        string, name of the existing column to rename.
    new : str
        string, new name of the column.
    
    Returns
    -------
    :class:`DataFrame`
        DataFrame with renamed column.
    
    Examples
    --------
    >>> df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["age", "name"])
    >>> df.withColumnRenamed('age', 'age2').show()
    +----+-----+
    |age2| name|
    +----+-----+
    |   2|Alice|
    |   5|  Bob|
    +----+-----+



In [0]:
df4=df3.withColumnRenamed('joindate','JoiningDate')
df4.show()

+---+----+---+-----------+
| id|name|Age|JoiningDate|
+---+----+---+-----------+
|  1|loki| 22| 2020-10-10|
|  2|gopi| 25| 2020-10-10|
+---+----+---+-----------+



In [0]:
# CREATE COMPLEX COLUMNS USING StructType & StructField
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
data1=[\
    (1,('sampathi ','lokesh'),25),\
    (2,('sampathi','gopal'),23)   ]
nestedschema=StructType([StructField('firstname',StringType()),\
                    StructField('lastname',StringType())
                    ])
schema=StructType([StructField('id',IntegerType()),\
                    StructField('name',nestedschema),\
                    StructField('age',IntegerType())])

df=spark.createDataFrame(data1,schema)
display(df)
df.printSchema()

id,name,age
1,"List(sampathi , lokesh)",25
2,"List(sampathi, gopal)",23


root
 |-- id: integer (nullable = true)
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)



In [0]:
# using ArrayType column & operations on it
    # accessing using index
    # combining values into single array
from pyspark.sql.types import ArrayType
from pyspark.sql.functions import col,array
data2=[\
    (1,['sampathi ','lokesh'],25),\
    (2,['sampathi','gopal'],23)   ]
schema2=StructType([StructField('id',IntegerType()),\
                    StructField('name',ArrayType(StringType())),\
                    StructField('age',IntegerType())])
df1=spark.createDataFrame(data2,schema2)
display(df1)


id,name,age
1,"List(sampathi , lokesh)",25
2,"List(sampathi, gopal)",23


In [0]:
#access data using index
df1.withColumn('firstname',df1.name[0]).show()
df1.withColumn('lastname',df1.name[1]).show()


+---+-------------------+---+---------+
| id|               name|age|firstname|
+---+-------------------+---+---------+
|  1|[sampathi , lokesh]| 25|sampathi |
|  2|  [sampathi, gopal]| 23| sampathi|
+---+-------------------+---+---------+

+---+-------------------+---+--------+
| id|               name|age|lastname|
+---+-------------------+---+--------+
|  1|[sampathi , lokesh]| 25|  lokesh|
|  2|  [sampathi, gopal]| 23|   gopal|
+---+-------------------+---+--------+



In [0]:
df1=df1.withColumn('firstname',df1.name[0]).withColumn('lastname',df1.name[1])
display(df1)



id,name,age,firstname,lastname
1,"List(sampathi , lokesh)",25,sampathi,lokesh
2,"List(sampathi, gopal)",23,sampathi,gopal


In [0]:
data3=[('samapthi','lokesh'),\
    ('sampathi','gopal')]
schema3=['firstname','lastname']
df2=spark.createDataFrame(data3,schema3)
display(df2)

firstname,lastname
samapthi,lokesh
sampathi,gopal


In [0]:
#combining diiferent values into single array
df2.withColumn('fullname',array(df2.firstname,df2.lastname)).show()

+---------+--------+------------------+
|firstname|lastname|          fullname|
+---------+--------+------------------+
| samapthi|  lokesh|[samapthi, lokesh]|
| sampathi|   gopal| [sampathi, gopal]|
+---------+--------+------------------+



In [0]:
#functions used in array type columns
    # explode
    # split
    # array
    # array_contains

In [0]:
from pyspark.sql.functions import explode,col,split,array_contains

In [0]:
# explode used to devide each elment of array into new rows of a array column
data=[('lokesh',['java','python','sql']),('gopi',['python','dotnet','dbms'])]
schema=['name','skills']
df=spark.createDataFrame(data,schema)
df=df.withColumn('skill',explode(col('skills')))
df.show(truncate=False)


+------+----------------------+------+
|name  |skills                |skill |
+------+----------------------+------+
|lokesh|[java, python, sql]   |java  |
|lokesh|[java, python, sql]   |python|
|lokesh|[java, python, sql]   |sql   |
|gopi  |[python, dotnet, dbms]|python|
|gopi  |[python, dotnet, dbms]|dotnet|
|gopi  |[python, dotnet, dbms]|dbms  |
+------+----------------------+------+



In [0]:
#split is used to split the string into array of values by using particular pattern
data1=[('lokesh','java,python,sql'),('gopi','python,dotnet,dbms')]
schema=['name','skills']
df=spark.createDataFrame(data1,schema)
df=df.withColumn('skills in array',split(col('skills'),','))
df.show(truncate=False)


+------+------------------+----------------------+
|name  |skills            |skills in array       |
+------+------------------+----------------------+
|lokesh|java,python,sql   |[java, python, sql]   |
|gopi  |python,dotnet,dbms|[python, dotnet, dbms]|
+------+------------------+----------------------+



In [0]:
#array_contains is used to check for particular value present in array or not
data=[('lokesh',['java','python','sql']),('gopi',['python','dotnet','dbms'])]
schema=['name','skills']
df=spark.createDataFrame(data,schema)
df.withColumn('has skill?',array_contains(col('skills'),'python')).show()
df.withColumn('has skill?',array_contains(col('skills'),'sql')).show()

#df.show(truncate=False)


+------+--------------------+----------+
|  name|              skills|has skill?|
+------+--------------------+----------+
|lokesh| [java, python, sql]|      true|
|  gopi|[python, dotnet, ...|      true|
+------+--------------------+----------+

+------+--------------------+----------+
|  name|              skills|has skill?|
+------+--------------------+----------+
|lokesh| [java, python, sql]|      true|
|  gopi|[python, dotnet, ...|     false|
+------+--------------------+----------+



In [0]:
#MAP type
# MAP type is same as dictionary type
from pyspark.sql.types import MapType,StringType,IntegerType,StructType,StructField
from pyspark.sql.functions import col

In [0]:
data=[('lokesh',{'telugu':25,'english':40}),('gopi',{'telugu':65,'english':48})]
schema=StructType([\
    StructField('name',StringType()),\
    StructField('marks',MapType(StringType(),IntegerType()))])
dfMAP=spark.createDataFrame(data,schema)
display(dfMAP)
dfMAP.printSchema()

name,marks
lokesh,"Map(english -> 40, telugu -> 25)"
gopi,"Map(english -> 48, telugu -> 65)"


root
 |-- name: string (nullable = true)
 |-- marks: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)



In [0]:
# accessing values of a map type
dfMAP=dfMAP.withColumn('Telugu',col('marks')['telugu']).withColumn('english',col('marks')['english'])
dfMAP.show(truncate=False)

+------+-----------------------------+------+-------+
|name  |marks                        |Telugu|english|
+------+-----------------------------+------+-------+
|lokesh|{english -> 40, telugu -> 25}|25    |40     |
|gopi  |{english -> 48, telugu -> 65}|65    |48     |
+------+-----------------------------+------+-------+



In [0]:
# functons used in map
#     explode
#     map_keys
#     map_values
from pyspark.sql.functions import explode,map_keys,map_values

In [0]:
#explode
dfMAP.select('name','marks',explode(col('marks'))).show()

+------+--------------------+-------+-----+
|  name|               marks|    key|value|
+------+--------------------+-------+-----+
|lokesh|{english -> 40, t...|english|   40|
|lokesh|{english -> 40, t...| telugu|   25|
|  gopi|{english -> 48, t...|english|   48|
|  gopi|{english -> 48, t...| telugu|   65|
+------+--------------------+-------+-----+



In [0]:
#map_keys & map_values
dfMAP.select('name','marks',map_keys(col('marks')),map_values(col('marks'))).show()

+------+--------------------+-----------------+-----------------+
|  name|               marks|  map_keys(marks)|map_values(marks)|
+------+--------------------+-----------------+-----------------+
|lokesh|{english -> 40, t...|[english, telugu]|         [40, 25]|
|  gopi|{english -> 48, t...|[english, telugu]|         [48, 65]|
+------+--------------------+-----------------+-----------------+



In [0]:
dfMAP.withColumn('keys',map_keys(col('marks'))).show()

+------+--------------------+------+-------+-----------------+
|  name|               marks|Telugu|english|             keys|
+------+--------------------+------+-------+-----------------+
|lokesh|{english -> 40, t...|    25|     40|[english, telugu]|
|  gopi|{english -> 48, t...|    65|     48|[english, telugu]|
+------+--------------------+------+-------+-----------------+

